In [1]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from IPython.display import Markdown, display

In [2]:
load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

# Check the key

if not openrouter_api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not openrouter_api_key.startswith("sk-or-v1-"):
    print("An API key was found, but it doesn't start sk-or-v1-; please check you're using the right key - see troubleshooting notebook")
elif openrouter_api_key.strip() != openrouter_api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

python-dotenv could not parse statement starting at line 9
python-dotenv could not parse statement starting at line 13


API key found and looks good so far!


In [3]:
# Connect to client libraries

# OpenRouter client (unified for all models)
openrouter = OpenAI(
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
)

In [4]:

# UI model names → OpenRouter model IDs
MODEL_MAP = {
    "gpt-5": "openai/gpt-5",
    "claude-sonnet-4-5": "anthropic/claude-sonnet-4.5",
    "grok-4": "x-ai/grok-4",
    "gemini-2.5-pro": "google/gemini-2.5-pro",
    "qwen2.5-coder": "qwen/qwen2.5-coder",
    "deepseek-coder-v2": "deepseek/deepseek-coder-v2",
    "gpt-oss-20b": "openai/gpt-oss-20b",
    "qwen3-coder-30b": "qwen/qwen3-coder-30b-a3b-instruct",
    "gpt-oss-120b": "openai/gpt-oss-120b",
}

# Models list
MODELS = list(MODEL_MAP.keys()) 
CURRENCIES = [
    "USD",  # US Dollar
    "NGN",  # Nigerian Naira
    "EUR",  # Euro
    "GBP",  # British Pound
    "CAD",  # Canadian Dollar
    "AUD",  # Australian Dollar
    "JPY",  # Japanese Yen
    "CNY",  # Chinese Yuan
    "INR",  # Indian Rupee
    "ZAR",  # South African Rand
    "CHF",  # Swiss Franc
    "AED",  # UAE Dirham
    "SGD",  # Singapore Dollar
]

In [5]:
def convert_with_model(amount, from_currency, to_currency, model_name):

    model_id = MODEL_MAP[model_name]

    prompt = f"""
Convert {amount} {from_currency} to {to_currency}.

Return ONLY in this JSON format:
{{
    "from": "{from_currency}",
    "to": "{to_currency}",
    "amount": {amount},
    "converted_amount": number,
    "exchange_rate": number
}}

Use the latest exchange rate.
"""

    response = openrouter.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": "You are a precise currency conversion assistant."},
            {"role": "user", "content": prompt},
        ],
        temperature=0,
        max_tokens=200,
    )

    return response.choices[0].message.content


In [ ]:
# Gradio UI
with gr.Blocks(title="AI Currency Converter (Model-powered)") as demo:

    gr.Markdown("# Convert Currency using Different AI Models")

    with gr.Row():
        amount = gr.Number(value=100, label="Amount")
        
    with gr.Row():

        from_currency = gr.Dropdown(
            choices=CURRENCIES,
            value="USD",
            label="From Currency",
        )

        to_currency = gr.Dropdown(
            choices=CURRENCIES,
            value="NGN",
            label="To Currency",
        )
    model = gr.Dropdown(
        choices=MODELS,
        value="gpt-5",
        label="Select Model",
    )

    convert_btn = gr.Button("Convert")

    output = gr.Code(
        label="Conversion Result",
        language="json",
    )

    convert_btn.click(
        fn=convert_with_model,
        inputs=[amount, from_currency, to_currency, model],
        outputs=output,
    )
    
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
